## Stitching

Earth Observation data is usually distributed as small scene files, with different projections, resolutions, etc. across data providers. eg: s3://modis-pds/, s3://sentinel-s2-l1c/ both have different file path layout and are in different projections.

Before using this data it's important to combine different scene files together, re-arranging bands and bringing the datasets to consistent resolution and projection.

This module aims to make it easy for anyone to download and combine EO scene files into either COGs or Zarr so that they can be visualized or analyzed easily.

The main class implemented is `dataset`.

### Using a grid file
1. Grid file is a kml or shapefile which contains a mapping of data provider's grid to world coordinates. 

2. It contains a `Name` column and a `geometry` which contains a mapping between x and y of satellite grid to bounding boxes. This file can be used to create a set of patterns, which then is used to search for scene files.

This is the advised method as using the grid file we can pinpoint exactly which files to download and stitch together

In [2]:
# Imports
import geopandas as gpd
import fiona
import re
import datetime

fiona.drvsupport.supported_drivers["kml"] = "rw"
fiona.drvsupport.supported_drivers["KML"] = "rw"

In [3]:
# The most important part of stitching is defining the source part to supply as an argument to `dataset` class
# An example is provided below

#### Figuring out source
Source is usually made up of three components and is derived from the full path of a single scene file

Eg: Path of single scene file

`s3://modis-pds/MCD43A4.006/01/08/2013160/MCD43A4.A2013160.h00v08.006.2016138043045_B07.TIF`

According to the documentation at https://docs.opendata.aws/modis-pds/readme.html the file path layout is

`/product/horizontal_grid/vertical_grid/date/DATA`

So for our example 
* `horizontal_grid` = 01 - {h}
* `vertical_grid` = 08 - {v}
* `date` = 2013160 - %Y%j according to python's standard format codes https://docs.python.org/3/library/datetime.html#format-codes
* `*_B07.TIF` -> For the file name. _You can use * or ? wildcards to select multiple files_

In [4]:
# So the source becomes
source = "s3://modis-pds/MCD43A4.006/{h}/{v}/%Y%j/*_B07.TIF"

In [5]:
# Modis Data is at a daily frequency so we create one COG per day
destination = "/Volumes/Data/spacetime-tools/final/modis-pds/%d-%m-%Y-b07.TIF"

In [6]:
# As an example we will use Albania's bounding box and get data for the month of January 2017 from s3://modis-pds/MCD43A4.006/
bbox = (19.3044861183, 39.624997667, 21.0200403175, 42.6882473822)
date_range = (datetime.datetime(2017, 1, 1), datetime.datetime(2017, 1, 31)) # (Start, End) - End date is inclusive

In [7]:
# Extracting h and v components from the grid. This is required for stitching
grid_fp = "../spacetime_tools/stitching/sample_data/sample_kmls/modis.kml"

# Loading the grid file
gdf = gpd.read_file(grid_fp)
print (gdf)

         Name Description                                           geometry
0    h:0 v:10              POLYGON Z ((-179.83326 -19.15932 0.00000, -178...
1     h:0 v:7              POLYGON Z ((-179.24909 9.98532 0.00000, -178.4...
2     h:0 v:8              POLYGON Z ((-178.19927 -0.00713 0.00000, -177....
3     h:0 v:9              POLYGON Z ((-179.99995 -0.00700 0.00000, -179....
4    h:1 v:10              POLYGON Z ((-170.13189 -19.99203 0.00000, -169...
..        ...         ...                                                ...
455   h:9 v:5              POLYGON Z ((-92.27929 29.97022 0.00000, -93.06...
456   h:9 v:6              POLYGON Z ((-85.05856 19.97777 0.00000, -85.51...
457   h:9 v:7              POLYGON Z ((-81.16972 9.98532 0.00000, -81.386...
458   h:9 v:8              POLYGON Z ((-79.94018 -0.00713 0.00000, -79.94...
459   h:9 v:9              POLYGON Z ((-81.17328 -9.99958 0.00000, -80.97...

[460 rows x 3 columns]


In [8]:
# To get the h and v components of file path layout we need to define a 
# lambda function which can extract h and v from the grid dataframe as a dictionary
# Example below
def fn(df_row):
    match = re.search(r"h:(\d*) v:(\d*)", df_row.Name)
    if match and match.groups():
        vars = match.groups()
        return {
            "h": f"{int(vars[0]):02d}",
            "v": f"{int(vars[1]):02d}",
        }

In [ ]:
# Importing the dataset class from spacetime_tools.stitching
from spacetime_tools.stitching.classes import dataset

In [ ]:
# AWS REGION
region = "us-west-2"

In [ ]:
# This initializes the dataset object
ds = dataset.DataSet("modis-pds", "s3", source, overwrite=True)

# Setting time bounds
ds.set_timebounds(date_range[0], date_range[1])

# Setting spatial bounds. We pass the bbox we are interested in,
# grid file path and the lambda function to extract h and v from the grid file
ds.set_spacebounds(bbox, grid_fp, fn)

In [ ]:
# Getting distinct bands. This will help us decide band arrangement when stitching scenes together
bands = ds.get_distinct_bands()
print (bands)

In [ ]:
# Downloading scene files
ds.sync()

In [ ]:
# Finally stitching them together with the band arrangement as below
ds.to_cog(
        destination,
        bands=[
            "Nadir_Reflectance_Band7",
        ],
    )